In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Problem Statment 

In this competition we are predicting the values of air pollution measurements over time, based on weather information (temperature and humidity) and the input values of 5 sensors.

The three target values we need to predict are: target_carbon_monoxide, target_benzene, and target_nitrogen_oxides

## Main Goal
In this notebook we will use average weighted ensemble model. In simple terms take output of different models else train different model by yourself and take the average of the model to create your submission file.Since this is my first time I tried using output of the following notebooks. There is lot of scope for improvemnt.
 
## Major credits 
### https://www.kaggle.com/junhyeok99/automl-pycaret  
### https://www.kaggle.com/maximkazantsev/tps-07-21-eda-catboost  
### https://www.kaggle.com/alexryzhkov/tps-lightautoml-baseline-with-pseudolabels  

## EDA 

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
plt.figure(figsize = (12, 8))
corr_train = train.corr()
sns.heatmap(corr_train, annot = True);

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['deg_C'], kde=True,color='red')
ax.set(xlabel='Degrees In Celsius', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['absolute_humidity'], kde=True,color='red')
ax.set(xlabel='Absolute Humidity', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['sensor_1'], kde=True,color='red')
ax.set(xlabel='Sensor 1', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['sensor_2'], kde=True,color='red')
ax.set(xlabel='Sensor 2', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['sensor_3'], kde=True,color='red')
ax.set(xlabel='Sensor 3', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['sensor_4'], kde=True,color='red')
ax.set(xlabel='Sensor 4', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['sensor_5'], kde=True,color='red')
ax.set(xlabel='Sensor 5', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['target_carbon_monoxide'], kde=True,color='blue')
ax.set(xlabel='Carbon Monoxide', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['target_benzene'], kde=True,color='blue')
ax.set(xlabel='Benzene', ylabel='Frequency')

In [ ]:
plt.figure(figsize=(20, 10))
ax = sns.histplot(train['target_nitrogen_oxides'], kde=True,color='blue')
ax.set(xlabel='Nitrogen Oxides', ylabel='Frequency')

## Conclusion
1) From the above graphs we can say Sensor 1,2,3,5 are Postive skewd (right skewed distribution).  
2) Degrees In Celsius,Absolute Humidity and Sensor 4 are Normally distributed  
3) All the target values are Right Skewd

In [ ]:
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(4,2) # create figure and axes
fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                    hspace=0.9, wspace=0.3)

df = pd.DataFrame(data={'Degree celsuis': train['deg_C'],
                    'Relative Humidity': train['relative_humidity'],
                    'Absolute Humidity': train['absolute_humidity'],
                    'Sensor 1 Data': train['sensor_1'],
                    'Sensor 2 Data': train['sensor_2'],
                    'Sensor 3 Data': train['sensor_3'],
                    'Sensor 4 Data': train['sensor_4'],
                    'Sensor 5 Data': train['sensor_5'],  
                       })

for i,el in enumerate(list(df.columns.values)):
    a = df.boxplot(el, ax=axes.flatten()[i])

#fig.delaxes(axes[1,1]) # remove empty subplot
plt.tight_layout() 

plt.show()

#### There are a lot of outliers in Degree Celsuis and Sensor 1,2,3,4,5

In [ ]:
sns.pairplot(train)

In [ ]:
sns.jointplot(
    data=train,
    x="target_carbon_monoxide", y="target_benzene",
)

In [ ]:
sns.jointplot(
    data=train,
    x="target_carbon_monoxide", y="target_nitrogen_oxides",
)

In [ ]:
sns.jointplot(
    data=train,
    x="target_benzene", y="target_nitrogen_oxides",
)

#### A strong perfect correlation exists between benzene and carbon monoxide and nitrogen oxide

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')
sns.scatterplot(x=train['date_time'], y= train['target_carbon_monoxide'])
plt.title("carbon monoxide distribution over time", size=20)

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')
sns.scatterplot(x=train['date_time'], y= train['target_benzene'])
plt.title("Benzene distribution over time", size=20)

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')
sns.scatterplot(x=train['date_time'], y= train['target_nitrogen_oxides'])
plt.title("Nitrogen Oxides distribution over time", size=20)

#### Nitrogen oxides concentration has increased & continue to have a high value since then.

In [ ]:
model_1 = pd.read_csv("../input/ensemble/lightautoml.csv")
model_2 = pd.read_csv("../input/ensemble/sub.csv")
model_3 = pd.read_csv("../input/ensemble/submission.csv")

In [ ]:
final_model = pd.concat([model_1,model_2,model_3]).groupby(level=0).mean()

In [ ]:
final_model['date_time'] = model_1['date_time']

In [ ]:
final_model= final_model[['date_time', 'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]

In [ ]:
final_model.to_csv('submission_average_weighted_blend.csv', index=False)

In [ ]:
final_model